In [1]:
import os
import cv2
import numpy as np
from osgeo import gdal
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pennylane as qml
from qiskit import Aer, execute, QuantumCircuit
from qiskit.circuit import QuantumCircuit, Parameter
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence

2024-02-29 10:02:28.982739: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-29 10:02:29.284942: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 10:02:29.285073: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 10:02:29.320923: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 10:02:29.407137: I tensorflow/core/platform/cpu_feature_guar

In [2]:
dataset_path = "/home/bharatforge/Selvin/EuroSAT_MS"

In [3]:
subdirs = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

# Initialize lists to store images and labels
images = []
labels = []

# Initialize a label counter
label = 0

# Loop through each subdirectory
for subdir in subdirs:
    # List all image files in the subdirectory
    image_files = [os.path.join(dataset_path, subdir, f) for f in os.listdir(os.path.join(dataset_path, subdir)) if f.endswith(".tif")]
    
    # Load images from the files
    for file in image_files:
        # Open the image file using GDAL
        dataset = gdal.Open(file)

        # Read all image channels into a list
        channels = [dataset.GetRasterBand(i + 1).ReadAsArray() for i in range(dataset.RasterCount)]

        # Stack the channels to create a multi-channel image
        image = np.stack(channels, axis=-1)

        # Resize the image to a fixed size (if needed)
        image = cv2.resize(image, (64, 64))  # Adjust the dimensions as per your requirements
        
        # Append the image and label to the lists
        images.append(image)
        labels.append(label)  # Use the label counter as the label
    
    # Increment the label counter for the next subdirectory
    label += 1

# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize pixel values to be between 0 and 1
images = images / 255.0

In [4]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [5]:
import tensorflow as tf
import pennylane as qml
from tensorflow.keras.layers import Layer

class QuantumBellman(Layer):
    def __init__(self, **kwargs):
        super(QuantumBellman, self).__init__(**kwargs)
        self.dev = qml.device("default.qubit", wires=4)

    def build(self, input_shape):
        super(QuantumBellman, self).build(input_shape)

    def call(self, inputs):
        # Convert inputs to tensor
        inputs = tf.convert_to_tensor(inputs)
        # Compute quantum circuit results
        output = tf.vectorized_map(self.quantum_func, inputs)
        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 16)

    def quantum_func(self, inputs):
        @qml.qnode(self.dev, interface='tf')
        def quantum_circuit(params):
            # Hadamard gate to put qubits in superposition
            qml.Hadamard(wires=0)

            # Successive CNOT gates to entangle qubits
            qml.CNOT(wires=[0, 1])
            qml.CNOT(wires=[1, 2])
            qml.CNOT(wires=[2, 3])

            # Rotation around the y-axis using the parameters θ
            for i in range(4):
                qml.RY(params[i], wires=i)

            # Reverse the CNOT operations
            qml.CNOT(wires=[2, 3])
            qml.CNOT(wires=[1, 2])
            qml.CNOT(wires=[0, 1])

            return qml.probs(wires=[0, 1, 2, 3])

        # Compute quantum circuit results
        probs = quantum_circuit(inputs)
        return probs

In [6]:
# Add layers to the model
model =Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 13)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(4, activation='relu'))
model.add(BatchNormalization())

# Add the Quantum layer
model.add(QuantumBellman())

model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(10, activation='softmax'))


2024-02-29 10:04:26.421428: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-29 10:04:26.581670: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-29 10:04:26.582151: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [7]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('Quantum_best_model_bellman.h5', save_best_only=True)

# Create an instance of the Adam optimizer with the specified learning rate


model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with callbacks
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping, model_checkpoint])

model.save('Quantum_best_model_bellman.h5')

class_names = ['SeaLake', 'Industrial', 'HerbaceousVegetation', 'PermanentCrop', 'AnnualCrop', 'Residential', 'Pasture', 'Forest', 'Highway', 'River']

# Evaluate the model


Epoch 1/30


2024-02-29 10:04:44.349891: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-29 10:04:44.594821: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-29 10:04:45.117257: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-29 10:04:47.289108: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f3b2abd5e10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-29 10:04:47.289132: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2024-02-29 10:04:47.298470: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1709181287.459705    5934 device_compiler.h:186] 

338/338 [==============================] - 20s 31ms/step - loss: 2.2286 - accuracy: 0.1899 - val_loss: 2.1535 - val_accuracy: 0.2778
Epoch 2/30
  4/338 [..............................] - ETA: 5s - loss: 2.0586 - accuracy: 0.3242

/home/bharatforge/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


338/338 [==============================] - 8s 24ms/step - loss: 1.9742 - accuracy: 0.3448 - val_loss: 1.8330 - val_accuracy: 0.4069
Epoch 3/30
338/338 [==============================] - 8s 24ms/step - loss: 1.8141 - accuracy: 0.3981 - val_loss: 1.7160 - val_accuracy: 0.4435
Epoch 4/30
338/338 [==============================] - 8s 24ms/step - loss: 1.6941 - accuracy: 0.4411 - val_loss: 1.6746 - val_accuracy: 0.4759
Epoch 5/30
338/338 [==============================] - 8s 24ms/step - loss: 1.5530 - accuracy: 0.5015 - val_loss: 1.3401 - val_accuracy: 0.5800
Epoch 6/30
338/338 [==============================] - 8s 24ms/step - loss: 1.4196 - accuracy: 0.5333 - val_loss: 1.2392 - val_accuracy: 0.5785
Epoch 7/30
338/338 [==============================] - 8s 24ms/step - loss: 1.3015 - accuracy: 0.5626 - val_loss: 1.2255 - val_accuracy: 0.5646
Epoch 8/30
338/338 [==============================] - 8s 24ms/step - loss: 1.2304 - accuracy: 0.5779 - val_loss: 1.0715 - val_accuracy: 0.6102
Epoch 9/30

In [8]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
training_loss, training_accuracy = model.evaluate(X_train, y_train)

print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

print()

print("Training loss:", training_loss)
print("Training accuracy:", training_accuracy)

675/675 [==============================] - 5s 7ms/step - loss: 0.3741 - accuracy: 0.9077
Test loss: 0.4843222498893738
Test accuracy: 0.8762962818145752

Training loss: 0.37412863969802856
Training accuracy: 0.9076851606369019


In [9]:
loaded_model = load_model("Quantum_best_model_bellman.h5", custom_objects={"QuantumBellman": QuantumBellman})

In [10]:
predictions = loaded_model.predict(X_test)

for i in range(len(predictions)):
    predicted_label = np.argmax(predictions[i])
    true_label = y_test[i]
    print(f"Predicted: {predicted_label}, True: {true_label}")

169/169 [==============================] - 2s 7ms/step
Predicted: 2, True: 2
Predicted: 6, True: 6
Predicted: 2, True: 2
Predicted: 5, True: 5
Predicted: 4, True: 4
Predicted: 1, True: 1
Predicted: 7, True: 7
Predicted: 7, True: 7
Predicted: 3, True: 8
Predicted: 9, True: 5
Predicted: 8, True: 5
Predicted: 7, True: 7
Predicted: 6, True: 6
Predicted: 1, True: 1
Predicted: 3, True: 3
Predicted: 7, True: 7
Predicted: 9, True: 9
Predicted: 6, True: 9
Predicted: 8, True: 1
Predicted: 3, True: 3
Predicted: 1, True: 1
Predicted: 5, True: 5
Predicted: 1, True: 1
Predicted: 5, True: 5
Predicted: 7, True: 7
Predicted: 8, True: 8
Predicted: 0, True: 0
Predicted: 2, True: 2
Predicted: 7, True: 7
Predicted: 6, True: 6
Predicted: 8, True: 5
Predicted: 4, True: 4
Predicted: 7, True: 7
Predicted: 8, True: 8
Predicted: 7, True: 7
Predicted: 2, True: 2
Predicted: 7, True: 7
Predicted: 1, True: 1
Predicted: 7, True: 7
Predicted: 5, True: 5
Predicted: 1, True: 1
Predicted: 8, True: 8
Predicted: 1, True: 1

In [11]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

# Convert predictions to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_labels))

169/169 [==============================] - 2s 7ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.85      0.88       375
           1       0.99      0.69      0.81       616
           2       0.94      0.91      0.92       605
           3       0.64      0.98      0.77       520
           4       0.99      0.99      0.99       611
           5       0.80      0.72      0.76       507
           6       0.94      0.98      0.96       473
           7       1.00      1.00      1.00       597
           8       0.75      0.71      0.73       591
           9       0.93      0.95      0.94       505

    accuracy                           0.88      5400
   macro avg       0.89      0.88      0.88      5400
weighted avg       0.89      0.88      0.88      5400



In [13]:
# custom_image_path = '/home/admin1/Selvin/BE/EuroSAT_MS/SeaLake/SeaLake_1063.tif'

# class_names = ['SeaLake', 'Industrial', 'HerbaceousVegetation', 'PermanentCrop', 'AnnualCrop', 'Residential', 'Pasture', 'Forest', 'Highway', 'River']


# dataset = gdal.Open(custom_image_path)

# channels = [dataset.GetRasterBand(i+1).ReadAsArray() for i in range(dataset.RasterCount)]

# image = np.stack(channels, axis=-1)

# image = cv2.resize(image, (64, 64))

# image = image/ 255.0

# image = np.expand_dims(image, axis=0)

# predictions = loaded_model.predict(image)

# predicted_label = np.argmax(predictions)

# predicted_class_name = class_names[predicted_label]

# # Print the predicted label
# print("Predicted Label:", predicted_class_name)